In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sample_data.recsysNN_utils import *
pd.set_option("display.precision", 1)

In [2]:
top10_df = pd.read_csv("/content/sample_data/content_top10_df.csv")
bygenre_df = pd.read_csv("/content/sample_data/content_bygenre_df.csv")
top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.1,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.0,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.1,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.9,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.2,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.8,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.1,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,4.0,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.9,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.2,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


In [3]:
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.4,10377
1,Adventure,234,3.4,8785
2,Animation,76,3.6,2588
3,Children,69,3.4,2472
4,Comedy,326,3.4,8911
5,Crime,139,3.5,4671
6,Documentary,13,3.8,280
7,Drama,342,3.6,10201
8,Fantasy,124,3.4,4468
9,Horror,56,3.2,1345


In [4]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


In [5]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [6]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [7]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


In [11]:
num_outputs = 32
tf.random.set_seed(1)

class L2NormalizeLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=1)

user_NN = tf.keras.models.Sequential([

  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
  L2NormalizeLayer() # Apply L2 normalization as a layer

])

item_NN = tf.keras.models.Sequential([

  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
  L2NormalizeLayer() # Apply L2 normalization as a layer

])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)


# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

#This defines the overall model using the Keras functional API. It takes the user and item input layers as inputs and the dot product output as the output.
# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_6        │ (None, 32)        │     40,864 │ input_layer_5[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_7        │ (None, 32)        │     41,376 │ input_layer_7[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ sequential_6[0][… │
│                     │                   │            │ sequential_7[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [13]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.1296
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1139
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.1082
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1051
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.1020
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0996
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0976
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0956
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0937
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0917
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0900
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0885
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0871
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0857
Epoch 15/30
1273/1273 ━━━━━━━

In [14]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0720
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0715
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0709
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0705
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0700
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0695
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0691
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0687
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.0684
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0680
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0677
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0673
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0669
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0666
Epoch 15/30
1273/1273 ━━━━━━

In [15]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0850


0.08140966296195984

In [16]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [19]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print(sorted_ypu, sorted_items, movie_dict)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
[[4.105981 ]
 [4.0426188]
 [4.025769 ]
 [4.0180387]
 [4.003633 ]
 [3.9531004]
 [3.9335413]
 [3.8934643]
 [3.875634 ]
 [3.872248 ]
 [3.866029 ]
 [3.8432357]
 [3.8204477]
 [3.8197663]
 [3.8068771]
 [3.7821298]
 [3.7720335]
 [3.7714887]
 [3.745108 ]
 [3.7301903]
 [3.7291758]
 [3.725262 ]
 [3.705552 ]
 [3.676138 ]
 [3.6707895]
 [3.6664987]
 [3.6515396]
 [3.6291585]
 [3.6270652]
 [3.6208422]
 [3.6094162]
 [3.5879436]
 [3.5795627]
 [3.5791333]
 [3.565909 ]
 [3.5637887]
 [3.5538821]
 [3.5469096]
 [3.5416453]
 [3.540601 ]
 [3.5216842]
 [3.5182283]
 [3.513761 ]
 [3.508801 ]
 [3.4976377]
 [3.4922836]
 [3.4882507]
 [3.4876938]
 [3.486546 ]
 [3.485077 ]
 [3.483251 ]
 [3.47855  ]
 [3.4772015]
 [3.4713364]
 [3.4616613]
 [3.4533172]
 [3.4467225]
 [3.4199557]
 [3.4139183]
 [3.4106152]
 [3.4047327]
 [3.3938897]
 [3.3937278]
 [3.3904228]
 [3.3751423]
 [3.3686078]
 [3.348448 ]
 [3.3474755]
 [3.328689 ]
 [3.3271065]
 [3.3257723]
 [3.322195 ]
 [3.305096 ]
 [3.3013146]

In [17]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


y_p,movie id,rating ave,title,genres
4.1,137857,3.6,The Jungle Book (2016),Adventure|Drama|Fantasy
4,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4,4896,3.8,Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),Adventure|Children|Fantasy
3.9,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
3.9,118696,3.4,The Hobbit: The Battle of the Five Armies (2014),Adventure|Fantasy
3.9,87232,3.8,X-Men: First Class (2011),Action|Adventure|Sci-Fi|Thriller
3.9,34405,3.9,Serenity (2005),Action|Adventure|Sci-Fi
